In [ ]:
import os, sys

# if os.path.exists('analysis'):
#     os.system('rm -rf analysis')

# !git clone https://github.com/element-fi/analysis.git

# sys.path.insert(1, os.getcwd())

In [ ]:
import json, numbers, math, time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sim import YieldSimulator, ElementPricingModel, Market, YieldSpacev2PricingModel, YieldSpacev2MinFeePricingModel

pd.set_option("float_format",'{:,.3f}'.format)

seed = 3
rng = np.random.default_rng(seed)

In [ ]:
config = {
    'step_size': 0.001,
    'min_fee': 0.,
    'max_fee': 0.5,
    't_min': 0.001,
    't_max': 1.0,
    'base_asset_price': 2500., # aka market price
    'min_target_liquidity': 100000.,
    'max_target_liquidity': 10000000.,
    'min_target_volume': 2e5,
    'max_target_volume': 6e5,
    'min_pool_apy': 0.5,
    'max_pool_apy': 50,
    'min_vault_age': 0.,
    'max_vault_age': 2,
    'min_vault_apy': 0.,
    'max_vault_apy': 10.,
    'precision': 8,
    'pricing_model_name': 'YieldSpace',
    'tokens': ['base', 'fyt'],
    'trade_direction': 'out',
    'days_until_maturity': 90,
    'num_trading_days': 90, # should be <= days_until_maturity
    'rng' : rng, # random number generator
}

In [ ]:
start_time = time.time()
num_runs = 0

pricing_model_list = ['Element', 'YieldSpacev2', 'YieldSpacev2MinFee']
vault_age_list = [1] #[0.1, 0.3, 0.5]
target_daily_volume_list = [5*1e5]
target_liquidity_list = [10*1e6]
fee_percent_list = [0.1]
#vault_apys = [[1,]*10 +[5,]*10 + [10,]*10 + [5,]*10 + [1,]*10,] # example variable vault apy
#vault_apys = [[i,]*config['num_trading_days'] for i in [3, 5, 9]] # example constant vault apy
# vault_apys = [[i,]*config['num_trading_days'] for i in [5]] # example constant vault apy
vault_apy_list = [[5,]*config['num_trading_days']] # constant vault apy

simulator = YieldSimulator(**config)
simulator.set_random_variables()

for pricing_model in pricing_model_list:
    for vault_age in vault_age_list:
        for vault_apy in vault_apy_list:
            for target_daily_volume in target_daily_volume_list:
                for target_liquidity in target_liquidity_list:
                    for fee_percent in fee_percent_list:
                        override_dict = {
                            'vault_apy': vault_apy,
                            'pricing_model_name': pricing_model,
                            'init_vault_age': vault_age,
                            'vault_apy': 5,
                            'init_pool_apy' : 5,
                            'target_daily_volume': target_daily_volume,
                            'target_liquidity': target_liquidity,
                            'fee_percent' : fee_percent,
                        }
                        simulator.run_simulation(override_dict)
                        num_runs += 1
                        print('run #{} completed'.format(num_runs))

end_time = time.time()
print(f'Total time for {num_runs} runs was {end_time-start_time:.3f} seconds; which is {(end_time-start_time)/num_runs:.3f} seconds per run')

In [ ]:
trades = pd.DataFrame.from_dict(simulator.analysis_dict)
display(trades.shape)
trades.head(5).T

In [ ]:
display(trades.tail(1).T)

In [ ]:
trades['time_diff'] = trades.time_until_end.diff()
trades['time_diff_shift'] = trades.time_until_end.shift(-1).diff()
trades.loc[len(trades)-1,'time_diff_shift'] = 1
trades['fee_in_bps'] = trades.fee / trades.out_without_fee * 100 * 100
trades.loc[(trades.time_diff>0) | (trades.time_diff_shift>0) | (trades.index == trades.index.max()),:]

In [ ]:
general_columns = ['run_number','model_name','num_orders','t_stretch','target_liquidity','target_daily_volume','pool_apy','fee_percent','init_vault_age','vault_apy','days_until_maturity','num_trading_days'] 
# create runs dataframe from the last trade in each run (using time_diff instead would pick the 1st)
runs = trades.loc[(trades.time_diff_shift>0),:].copy().loc[:,general_columns]
# variables that change per trade represent the last value in the run, rename the useful ones
runs.rename(columns={'current_apy':'ending_apy','num_orders':'total_orders'}, inplace=True)
# general_columns=runs.columns.to_list()
runs.set_index('run_number', inplace=True)
display(runs.T)

In [ ]:
do_not_aggregate = ['run_number','model_name','num_orders']
print('inspect standard deviations to know what to aggregate or not')
dfg_std = trades.groupby(general_columns[0])[[x for x in general_columns if x not in do_not_aggregate]].std()
display(dfg_std.T) # check if these are really constant

In [ ]:
trade_columns = list(set(trades.columns) - set(general_columns) - set(do_not_aggregate))
display(trade_columns)

In [ ]:
# do mean things
mean_columns = ['time_until_end','init_price_per_share','base_asset_price','spot_price','out_without_fee_slippage','x_reserves',\
    'out_without_fee','fee','trade_amount','out_with_fee','day','fee_in_bps','y_reserves','total_supply']
dfg_mean = trades.groupby(['run_number'])[mean_columns].mean()
dfg_mean.columns = ['mean_'+col for col in dfg_mean.columns]

# do sum things
sum_columns = ['fee','out_with_fee','out_without_fee','out_without_fee_slippage','trade_amount']
dfg_sum = trades.groupby(['run_number'])[sum_columns].sum()
dfg_sum.columns = ['sum_'+col for col in dfg_sum.columns]

dfg = pd.concat([runs, dfg_mean, dfg_sum], axis=1)
dfgt = dfg.T
display(dfgt)
dfgt.to_csv('dfgt.csv')

In [ ]:
display(trades.columns)
# display(trades.loc[trades.run_number==1,'num_orders'])

In [ ]:
debug = True
# dfs = []
# oldIndex = []

# calculat derived variables across runs
trades['total_liquidity']=trades.loc[:,'x_reserves']*trades.loc[:,'base_asset_price']+trades.loc[:,'y_reserves']*trades.loc[:,'base_asset_price']*trades.loc[:,'spot_price']
trades['trade_volume_usd']=trades.loc[:,'out_with_fee']*trades.loc[:,'base_asset_price']
# df['pu']=df.loc[:,'spot_price']*df.loc[:,'input.u']/df.loc[0,'spot_price'] # this is apr (includes compounding)
trades['pr']=((trades.loc[:,'spot_price']-trades.loc[0,'spot_price'])/1) # this is APR (does not include compounding)
trades['pu']=(trades.loc[:,'pr']+1)*trades.loc[:,'init_price_per_share'] # this is APR (does not include compounding)
trades['direction'] = 'out' # we know direction is always out
display(trades.loc[:,['day','num_orders','trade_amount','spot_price','pr','pu']])

# separate results into different DFs based on run
# for (model_name,yba,g,target_liquidity,target_daily_volume) in run_matrix:
#   newIndex = (df['init.market_price']==yba["market_price"]) & (df['init.apr']==yba["apr"]) & (df['init.percent_fee']==g) & (df['init.days_until_maturity']==yba["days_until_maturity"]) & (df['init.target_liquidity']==target_liquidity) & (df['init.target_daily_volume']==target_daily_volume)
#   if len(oldIndex)==0 or not all(newIndex==oldIndex):
#     dfs.append(df[ newIndex ].reset_index(drop=True))
#     oldIndex = newIndex

numPlots = 5
for run_id in range(0,trades.run_number.max()+1):
  # TODO: is trade_number identical to num_orders?
  df = trades.loc[trades.run_number==run_id,:].reset_index().rename(columns = {'index':'trade_number'})
  # calculate derived variables per run
  # df['fee_sum'] = df.fee.cumsum()

  fig, ax = plt.subplots(ncols=1, nrows=numPlots,gridspec_kw = {'wspace':0, 'hspace':0, 'height_ratios':np.ones(numPlots)})
  fig.patch.set_facecolor('white')   # set fig background color to white
  df_fees_volume = trades.groupby(['day','model_name']).agg({'trade_volume_usd':['sum']\
                                  ,'fee':['mean','std','min','max','sum']\
                                })
  df_fees_by_trade_type = trades.groupby(['model_name','direction','token_in']).agg({'trade_volume_usd':['sum']\
                                  ,'num_orders':['count']\
                                  ,'fee_in_bps':['mean','std','min','max','sum']\
                                  # ,'input.amount':['mean','std','min','max','sum']\
                                  # ,'output.slippage':['mean','std','min','max','sum']\
                                  # ,'output.fee':['mean','std','min','max','sum']\
                                }).rename(columns={'num_orders_sum':'trade_number_sum'})
  if debug: display(df_fees_by_trade_type)
                            
  df_fees_volume.columns = ['_'.join(col).strip() for col in df_fees_volume.columns.values]
  df_fees_volume = df_fees_volume.reset_index()

  for model in df_fees_volume.model_name.unique():
    ax[0] = df_fees_volume.loc[df_fees_volume.model_name==model,:].plot(x="day", y="fee_sum",figsize=(24,18),ax=ax[0],label=model)
  ax[0].set_xlabel("")
  ax[0].set_ylabel("Fees (US Dollars)",fontsize=18)
  ax[0].tick_params(axis = "both", labelsize=18)
  ax[0].grid(visible=True,linestyle='--', linewidth='1', color='grey',which='both',axis='y')
  ax[0].xaxis.set_ticklabels([])
  title = "Fees Collected Per Day Until Maturity\nAPY: {:.2f}%, Time Stretch: {:.2f}, Maturity: {:} days\n\
          Target Liquidity: {:.2f}, Target Daily Volume: {:.2f}, Percent Fees: {:.2f}%"\
    .format(df['pool_apy'][0],df['t_stretch'][0],df['days_until_maturity'][0]\
      ,df["target_daily_volume"][0],df["target_liquidity"][0],df["fee_percent"][0])
  ax[0].set_title(title,fontsize=20)
  ax[0].legend(fontsize=18)

  currentPlot = 1
  df_to_display = pd.DataFrame()
  for model in df_fees_volume.model_name.unique(): # for each model (per run?)
    ax[currentPlot] = trades.loc[trades.model_name==model,:]\
      .plot(x="num_orders",y="pool_apy",figsize=(24,18),ax=ax[currentPlot],label=model)
    df_to_display = pd.concat([df_to_display,trades.loc[trades.model_name==model,:].head(1)])
  df_to_display=df_to_display.set_index('model_name',drop=True)
  df_to_display.loc['diff']=[df_to_display.iloc[1,i]-df_to_display.iloc[0,i] if isinstance(df_to_display.iloc[0,i],numbers.Number) else df_to_display.iloc[0,i] for i in range(0,df_to_display.shape[1])]
  df_to_display.loc['ratio']=[df_to_display.iloc[1,i]/df_to_display.iloc[0,i] if isinstance(df_to_display.iloc[0,i],numbers.Number) else df_to_display.iloc[0,i] for i in range(0,df_to_display.shape[1])]
  if debug: display(df_to_display.loc[:,(df_to_display.iloc[0,:].values!=df_to_display.iloc[1,:].values) | (df_to_display.columns.isin(['price_per_share','init_price_per_share']))].T)

  ax[currentPlot] = df.plot(x="trade_number",y="vault_apy",figsize=(24,18),ax=ax[currentPlot],label='vault_apy {}→{}'.format(df.loc[:,'vault_apy'].values[0],df.loc[:,'vault_apy'].values[-1]))
  ax[currentPlot].set_xlabel("")
  ax[currentPlot].set_ylabel("apy",fontsize=18)
  ax[currentPlot].tick_params(axis = "both", labelsize=18)
  ax[currentPlot].grid(visible=True,linestyle='--', linewidth='1', color='grey',which='both',axis='y')
  ax[currentPlot].xaxis.set_ticklabels([])
  ax[currentPlot].legend(fontsize=18)

  currentPlot = 2 # c and u
  ax[currentPlot] = df.plot(x="trade_number",y="price_per_share",figsize=(24,18),ax=ax[currentPlot],label='c {:,.3f}→{:,.3f} r={:.3%}'.format(df.loc[:,'price_per_share'].values[0],df.loc[:,'price_per_share'].values[-1],df.loc[:,'price_per_share'].values[-1]/df.loc[:,'price_per_share'].values[0]-1))
  ax[currentPlot] = df.plot(x="trade_number",y="init_price_per_share",figsize=(24,18),ax=ax[currentPlot],label='u')
  ax[currentPlot] = df.plot(x="trade_number",y="pu",figsize=(24,18),ax=ax[currentPlot],label='p {:,.3f}→{:,.3f} r={:.3%}'.format(df.loc[:,'pu'].values[0],df.loc[:,'pu'].values[-1],df.loc[:,'pu'].values[-1]/df.loc[:,'pu'].values[0]-1))
  # ax2 = df.loc[df.model_name==model,:].plot(secondary_y=True,x="output.trade_number",y='spot_price',figsize=(24,18),ax=ax[currentPlot],label='p')
  ax[currentPlot].set_ylabel("Price Per Share",fontsize=18)
  ax[currentPlot].tick_params(axis = "both", labelsize=18)
  ax[currentPlot].grid(visible=True,linestyle='--', linewidth='1', color='grey',which='both',axis='y')
  ax[currentPlot].xaxis.set_ticklabels([])

  currentPlot = 3
  for model in df_fees_volume.model_name.unique():
    ax[currentPlot] = df_fees_volume.loc[df_fees_volume.model_name==model,:]\
      .plot(kind='line',x="day", y="trade_volume_usd_sum",ax=ax[currentPlot],label=model)
  ax[currentPlot].set_xlabel("Day",fontsize=18)
  ax[currentPlot].set_ylabel("Volume (US Dollars)",fontsize=18)
  ax[currentPlot].tick_params(axis = "both", labelsize=12)
  ax[currentPlot].grid(visible=True,linestyle='--', linewidth='1', color='grey',which='both',axis='y')
  ax[currentPlot].legend(fontsize=18)
  ax[currentPlot].ticklabel_format(style='plain',axis='y')
  fig.subplots_adjust(wspace=None, hspace=None)

  currentPlot = 4
  for model in df_fees_volume.model_name.unique():
    ax[currentPlot] = trades.loc[:,:]\
      .plot(kind='line',x="day", y=["x_reserves","y_reserves"],ax=ax[currentPlot],label=[model+'x',model+'y']) # .plot(kind='line',x="day", y="total_liquidity",ax=ax[currentPlot],label=model)
  # ax[currentPlot-2].plot(ax[currentPlot-2].lines[0].get_xdata()\
  #   ,ax[currentPlot].lines[0].get_ydata()/ax[currentPlot].lines[1].get_ydata()*ax[currentPlot-2].lines[0].get_ydata()[0]\
  #     ,label='liquidityDiff')
  ax[currentPlot-2].legend(fontsize=18)
  ax[currentPlot].set_xlabel("Day",fontsize=18)
  ax[currentPlot].set_ylabel("Liquidity (US Dollars)",fontsize=18)
  ax[currentPlot].tick_params(axis = "both", labelsize=12)
  ax[currentPlot].grid(visible=True,linestyle='--', linewidth='1', color='grey',which='both',axis='y')
  ax[currentPlot].legend(fontsize=18)
  ax[currentPlot].ticklabel_format(style='plain',axis='y')
  fig.subplots_adjust(wspace=None, hspace=None)

  plt.show()
  os.makedirs(os.getcwd()+"\\figures", exist_ok=True)
  fileName = "{}\\figures\chart{}.png".format(os.getcwd(),run_id+1)
  fig.savefig(fname=fileName,bbox_inches='tight')